# <center> • **NLP and Text Mining** </center>
### <center> _____________• Assignment 04: Emotion Detection in Arabic Text Using Classical and Deep Learning Techniques_____________ </center>
### <center> _____________• By:_____________ </center>
#### <center> _____________○ Belal Khaled ~ 2136873_____________ </center>
#### <center> _____________○ Yaseen Naser ~ 2130397_____________ </center>
#### <center> _____________○ Abdulrahman Abuhani ~ 2132462_____________ </center>
# ـــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــ

# <center> • EmotionalTone Arabic Dataset: </center>
## <center> ○ ____________https://github.com/amrmalkhatib/Emotional-Tone/tree/master____________</center>
## <center> ○ ____________The best results over a set of eight emotions were obtained using a complement Naïve Bayes algorithm with an overall accuracy of 68.12 %.____________</center>

# ـــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــ

# • Importing All Libraries we Need:

In [3]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# ـــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــ

# • Load the Dataset:

In [7]:
data = pd.read_csv("Emotional-Tone-Dataset.csv")

# ـــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــ

# • Explore the Dataset:

In [11]:
data.head()

,ID,TWEET,LABEL
0,1,الاوليمبياد الجايه هكون لسه ف الكليه ..,none
1,2,عجز الموازنه وصل ل93.7 % من الناتج المحلي يعني...,anger
2,3,كتنا نيله ف حظنا الهباب xD,sadness
3,4,جميعنا نريد تحقيق اهدافنا لكن تونس تالقت في حر...,joy
4,5,الاوليمبياد نظامها مختلف .. ومواعيد المونديال ...,none


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10065 entries, 0 to 10064
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      10065 non-null  int64 
 1    TWEET  10064 non-null  object
 2    LABEL  10065 non-null  object
dtypes: int64(1), object(2)
memory usage: 236.0+ KB


In [15]:
data.describe()

,ID
count,10065.000000
mean,5033.000000
std,2905.659564
min,1.000000
25%,2517.000000
50%,5033.000000
75%,7549.000000
max,10065.000000


In [17]:
data.columns

Index(['ID', ' TWEET', ' LABEL'], dtype='object')

In [19]:
data.columns = data.columns.str.strip()
data.columns

Index(['ID', 'TWEET', 'LABEL'], dtype='object')

# ـــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــ

# • Clean the Text:

In [23]:
# • Remove Unwanted Characters Like Punctuation, Numbers, and Non-Arabic Letters.
# • Remove Diacritics (Short Vowels) to Normalize Arabic Text.
# • Remove Stop Words (Common Words Like "و", "في", "على" That Do Not Contribute to Emotions).

#Function to Clean Arabic Text:
def clean_text(text):
    #Check If Text is Not a String:
    if not isinstance(text, str):  
        return ""
    #Removing Punctuation and Numbers:
    text = re.sub(r'[^\u0600-\u06FF\s]', '', text)  
    #Removing Diacritics:
    text = re.sub(r'[\u064B-\u065F]', '', text)
    #Removing Stop Words:
    arabic_stopwords = set(stopwords.words('arabic'))
    text = ' '.join([word for word in text.split() if word not in arabic_stopwords])
    return text

#Applying the clean_text Function to the 'TWEET' Column (Adding New Column 'cleaned_TWEET'):
data['cleaned_TWEET'] = data['TWEET'].apply(clean_text)

#Displaying 'TWEET' Column and 'cleaned_TWEET' Column:
data[["TWEET", "cleaned_TWEET"]].head()

,TWEET,cleaned_TWEET
0,الاوليمبياد الجايه هكون لسه ف الكليه ..,الاوليمبياد الجايه هكون لسه الكليه
1,عجز الموازنه وصل ل93.7 % من الناتج المحلي يعني...,عجز الموازنه وصل الناتج المحلي يعني لسه اقل نف...
2,كتنا نيله ف حظنا الهباب xD,كتنا نيله حظنا الهباب
3,جميعنا نريد تحقيق اهدافنا لكن تونس تالقت في حر...,جميعنا نريد تحقيق اهدافنا تونس تالقت حراسه المرمي
4,الاوليمبياد نظامها مختلف .. ومواعيد المونديال ...,الاوليمبياد نظامها مختلف ومواعيد المونديال مكا...


# ـــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــ

# • Tokenize the Arabic Text:

In [26]:
#Tokenizing the 'cleaned_TWEET' Column (Adding New Column 'tokenized_TWEET'):
data['tokenized_TWEET'] = data['cleaned_TWEET'].apply(word_tokenize)

#Displaying 'cleaned_TWEET' Column and 'tokenized_TWEET' Column:
data[["cleaned_TWEET", "tokenized_TWEET"]].head()

,cleaned_TWEET,tokenized_TWEET
0,الاوليمبياد الجايه هكون لسه الكليه,"[الاوليمبياد, الجايه, هكون, لسه, الكليه]"
1,عجز الموازنه وصل الناتج المحلي يعني لسه اقل نف...,"[عجز, الموازنه, وصل, الناتج, المحلي, يعني, لسه..."
2,كتنا نيله حظنا الهباب,"[كتنا, نيله, حظنا, الهباب]"
3,جميعنا نريد تحقيق اهدافنا تونس تالقت حراسه المرمي,"[جميعنا, نريد, تحقيق, اهدافنا, تونس, تالقت, حر..."
4,الاوليمبياد نظامها مختلف ومواعيد المونديال مكا...,"[الاوليمبياد, نظامها, مختلف, ومواعيد, المونديا..."


# ـــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــ

# • Normalize the Words:

In [29]:
# • Handle Word Variants, Such as:
    # • Variants of "أ", "إ", and "آ" → Normalize to "ا".
    # • Variants of "ة" → Normalize to "ه".
    # • Variants of repeated characters.

def normalize_text(text):
    text = re.sub(r'[إأآا]', 'ا', text)  
    text = re.sub(r'ة', 'ه', text)
    text = re.sub(r'(.)\1+', r'\1', text) 
    return text

#Applying Normalization to the 'tokenized_TWEET' Column (Adding New Column 'normalized_TWEET'):
data['normalized_TWEET'] = data['tokenized_TWEET'].apply(lambda x: [normalize_text(word) for word in x])

#Displaying 'tokenized_TWEET' Column and 'normalized_TWEET' Column:
data[["tokenized_TWEET", "normalized_TWEET"]].head()

,tokenized_TWEET,normalized_TWEET
0,"[الاوليمبياد, الجايه, هكون, لسه, الكليه]","[الاوليمبياد, الجايه, هكون, لسه, الكليه]"
1,"[عجز, الموازنه, وصل, الناتج, المحلي, يعني, لسه...","[عجز, الموازنه, وصل, الناتج, المحلي, يعني, لسه..."
2,"[كتنا, نيله, حظنا, الهباب]","[كتنا, نيله, حظنا, الهباب]"
3,"[جميعنا, نريد, تحقيق, اهدافنا, تونس, تالقت, حر...","[جميعنا, نريد, تحقيق, اهدافنا, تونس, تالقت, حر..."
4,"[الاوليمبياد, نظامها, مختلف, ومواعيد, المونديا...","[الاوليمبياد, نظامها, مختلف, ومواعيد, المونديا..."


# ـــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــ

# • Bag-of-Words (BoW):

In [32]:
#Joining the Tokenized Words Back Into Sentences:
data['normalized_TWEET'] = data['normalized_TWEET'].apply(lambda x: ' '.join(x))

#Initializing the CountVectorizer:
BOW_vectorizer = CountVectorizer(max_features=5000)

#Fitting and Transforming the Text Data:
BOW_features = BOW_vectorizer.fit_transform(data['normalized_TWEET'])

#Converting to DataFrame for Better Readability:
BOW_df = pd.DataFrame(BOW_features.toarray(), columns=BOW_vectorizer.get_feature_names_out())

BOW_df.head()

,اب,ابتديت,ابتسامتك,ابتسامه,ابتسم,ابحث,ابدا,ابداع,ابراهيم,ابرز,...,يوفقه,يوفقهم,يول,يوم,يوما,يومه,يومها,يومين,يونس,گل
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# ـــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــ

# • TF-IDF:

In [35]:
#Initializing the TfidfVectorizer:
TFIDF_vectorizer = TfidfVectorizer(max_features=5000)

#Fitting and Transforming the Text Data:
TFIDF_features = TFIDF_vectorizer.fit_transform(data['normalized_TWEET'])

# Convert to DataFrame for better readability
TFIDF_df = pd.DataFrame(TFIDF_features.toarray(), columns=TFIDF_vectorizer.get_feature_names_out())

TFIDF_df.head()

,اب,ابتديت,ابتسامتك,ابتسامه,ابتسم,ابحث,ابدا,ابداع,ابراهيم,ابرز,...,يوفقه,يوفقهم,يول,يوم,يوما,يومه,يومها,يومين,يونس,گل
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# ـــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــ

# • Word2Vec (Pre-trained Models like AraVec or fastText):

In [38]:
# • AraVec Used Here:
try:
    #Loading the Model Using Word2Vec:
    word2vec_model = Word2Vec.load(
        r'C:\Users\BKSH\OneDrive - Hashemite University\Desktop\BKSH HU\Natural Language Processing\Assignments\Assignment 4\Models\Twt-SG'
    )
    print("Model loaded successfully!")
except Exception as e:
    print("Error loading model:", e)

def average_word2vec(tokens, model):
    #Filtering Out Words Not in the Vocabulary:
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    if len(vectors) == 0:
        return np.zeros(model.wv.vector_size)
    return np.mean(vectors, axis=0)

#Applying the Function to Get Sentence Embeddings for Each Row:
data['sentence_embeddings'] = data['normalized_TWEET'].apply(
    lambda x: average_word2vec(x.split(), word2vec_model)
)

# • Example: ○ Accessing the Vector for the First Sentence:
data['sentence_embeddings'].iloc[0]

Model loaded successfully!


array([-9.59469937e-03,  1.57624990e-01, -2.03452021e-01,  1.17573902e-01,
        1.06548741e-01, -9.86032709e-02, -6.60574958e-02,  3.56602371e-02,
       -9.03370902e-02,  7.30422288e-02, -1.44537613e-01,  2.68261373e-01,
        5.66276796e-02,  3.77053432e-02,  6.25307709e-02,  6.56563193e-02,
        2.02920884e-01,  2.83338696e-01,  1.41982645e-01,  2.84037173e-01,
       -4.19083744e-01,  4.45718281e-02,  1.37060240e-01, -2.91097467e-03,
       -1.24953575e-02,  3.20910923e-02,  1.06452722e-02,  2.42769405e-01,
        1.54280469e-01, -1.27435476e-01, -1.04157425e-01,  1.28446892e-01,
        1.70553580e-01, -5.80303837e-03,  2.42562100e-01, -1.28271906e-02,
        1.57604814e-01, -1.53687134e-01,  4.56495345e-01, -8.34420044e-03,
        2.05491967e-02,  1.15231551e-01,  4.25996967e-02, -2.58710589e-02,
       -1.07627556e-01, -9.87990573e-02,  1.21960461e-01, -3.84536397e-04,
        1.10248491e-01,  9.21784788e-02,  2.91089956e-02,  2.40413785e-01,
        8.10044259e-02,  

# ـــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــ

# • Train-Test Split:

In [46]:
#Converting Word2Vec embeddings to a NumPy array
X_word2vec = np.vstack(data['sentence_embeddings'].values)

#Target label
y = data['LABEL']

#Splitting for BoW
X_train_bow, X_test_bow, y_train, y_test = train_test_split(BOW_df, y, test_size=0.2, random_state=42)

#Splitting for TF-IDF
X_train_tfidf, X_test_tfidf, _, _ = train_test_split(TFIDF_df, y, test_size=0.2, random_state=42)

#Splitting for Word2Vec
X_train_w2v, X_test_w2v, _, _ = train_test_split(X_word2vec, y, test_size=0.2, random_state=42)

# ـــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــ

# • Train and Evaluate Classifiers:

In [211]:
#Initializing the classifiers
classifiers = {
    "Naive Bayes": MultinomialNB(),
    "SVM (RBF Kernel)": SVC(kernel='rbf', probability=True),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "AdaBoost": AdaBoostClassifier()
}

#Evaluating each classifier on all three representations
for name, clf in classifiers.items():
    print(f"--- {name} ---")
    
    #BoW
    clf.fit(X_train_bow, y_train)
    y_pred_bow = clf.predict(X_test_bow)
    print("BoW Accuracy:", accuracy_score(y_test, y_pred_bow))
    print(classification_report(y_test, y_pred_bow))
    
    #TF-IDF
    clf.fit(X_train_tfidf, y_train)
    y_pred_tfidf = clf.predict(X_test_tfidf)
    print("TF-IDF Accuracy:", accuracy_score(y_test, y_pred_tfidf))
    print(classification_report(y_test, y_pred_tfidf))
    
    #Skipping Word2Vec for MultinomialNB ==> due to its inability to process negative values.
    if name == "Naive Bayes":
        print("Skipping Word2Vec for Naive Bayes due to negative values.")
        continue
    
    #Word2Vec
    clf.fit(X_train_w2v, y_train)
    y_pred_w2v = clf.predict(X_test_w2v)
    print("Word2Vec Accuracy:", accuracy_score(y_test, y_pred_w2v))
    print(classification_report(y_test, y_pred_w2v))

--- Naive Bayes ---
BoW Accuracy: 0.6229508196721312
              precision    recall  f1-score   support

       anger       0.67      0.62      0.65       276
        fear       0.77      0.89      0.83       259
         joy       0.61      0.34      0.44       268
        love       0.63      0.72      0.67       250
        none       0.53      0.89      0.67       307
     sadness       0.48      0.24      0.32       258
    surprise       0.54      0.40      0.46       201
    sympathy       0.72      0.86      0.78       194

    accuracy                           0.62      2013
   macro avg       0.62      0.62      0.60      2013
weighted avg       0.62      0.62      0.60      2013

TF-IDF Accuracy: 0.6105315449577745
              precision    recall  f1-score   support

       anger       0.63      0.67      0.65       276
        fear       0.78      0.86      0.82       259
         joy       0.58      0.32      0.41       268
        love       0.63      0.68      0.65

C:\Users\BKSH\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


BoW Accuracy: 0.41033283656234476
              precision    recall  f1-score   support

       anger       0.24      0.73      0.36       276
        fear       0.99      0.87      0.92       259
         joy       0.15      0.20      0.17       268
        love       0.85      0.28      0.43       250
        none       0.47      0.26      0.33       307
     sadness       0.00      0.00      0.00       258
    surprise       0.57      0.20      0.30       201
    sympathy       0.64      0.79      0.71       194

    accuracy                           0.41      2013
   macro avg       0.49      0.42      0.40      2013
weighted avg       0.47      0.41      0.39      2013



C:\Users\BKSH\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


TF-IDF Accuracy: 0.40735221063089916
              precision    recall  f1-score   support

       anger       0.24      0.76      0.37       276
        fear       0.99      0.85      0.92       259
         joy       0.15      0.22      0.18       268
        love       0.80      0.32      0.45       250
        none       0.48      0.21      0.29       307
     sadness       0.14      0.00      0.01       258
    surprise       0.65      0.15      0.24       201
    sympathy       0.64      0.80      0.71       194

    accuracy                           0.41      2013
   macro avg       0.51      0.41      0.40      2013
weighted avg       0.50      0.41      0.39      2013



C:\Users\BKSH\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Word2Vec Accuracy: 0.4972677595628415
              precision    recall  f1-score   support

       anger       0.54      0.53      0.53       276
        fear       0.63      0.68      0.65       259
         joy       0.40      0.21      0.28       268
        love       0.50      0.64      0.56       250
        none       0.53      0.79      0.64       307
     sadness       0.29      0.15      0.20       258
    surprise       0.36      0.27      0.31       201
    sympathy       0.48      0.65      0.55       194

    accuracy                           0.50      2013
   macro avg       0.47      0.49      0.47      2013
weighted avg       0.47      0.50      0.47      2013



# ـــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــ

# • Encode the Labels:

In [49]:
#Encoding the target labels
encoder = LabelEncoder()
y_train_encoded = encoder.fit_transform(y_train)
y_test_encoded = encoder.transform(y_test)

#Converting to one-hot encoding
y_train_onehot = to_categorical(y_train_encoded)
y_test_onehot = to_categorical(y_test_encoded)

# ـــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــ

# • Define the FNN Architecture:

In [53]:
def create_fnn_model(input_dim, num_classes):
    model = Sequential([
        Dense(128, activation='relu', input_dim=input_dim),
        Dropout(0.3),  
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')  
    ])
    
    #Compiling the model
    model.compile(optimizer='adam', 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
    
    return model

# ـــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــ

# • Train the FNN:

In [57]:
# • Training with TF-IDF:

#Creating the model for TF-IDF input
tfidf_model = create_fnn_model(input_dim=X_train_tfidf.shape[1], num_classes=y_train_onehot.shape[1])

#Training the model
history_tfidf = tfidf_model.fit(
    X_train_tfidf, y_train_onehot,
    validation_data=(X_test_tfidf, y_test_onehot),
    epochs=10,
    batch_size=32,
    verbose=1
)

C:\Users\BKSH\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
252/252 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.2598 - loss: 1.9580 - val_accuracy: 0.5941 - val_loss: 1.2452
Epoch 2/10
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6696 - loss: 1.0131 - val_accuracy: 0.6379 - val_loss: 1.0889
Epoch 3/10
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8062 - loss: 0.6279 - val_accuracy: 0.6329 - val_loss: 1.1440
Epoch 4/10
252/252 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8858 - loss: 0.4123 - val_accuracy: 0.6289 - val_loss: 1.2572
Epoch 5/10
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9154 - loss: 0.2896 - val_accuracy: 0.6150 - val_loss: 1.4120
Epoch 6/10
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9458 - loss: 0.2054 - val_accuracy: 0.6051 - val_loss: 1.5474
Epoch 7/10
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9522 - loss: 0.1637 - val_accuracy: 0.6036 - val_loss: 1.6980
Epoch 8/10
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9652 - loss: 0.1272 - val_accuracy: 0.

In [58]:
# • Training with Word2Vec:

#Creating the model for Word2Vec input
w2v_model = create_fnn_model(input_dim=X_train_w2v.shape[1], num_classes=y_train_onehot.shape[1])

#Training the model
history_w2v = w2v_model.fit(
    X_train_w2v, y_train_onehot,
    validation_data=(X_test_w2v, y_test_onehot),
    epochs=10,
    batch_size=32,
    verbose=1
)

Epoch 1/10


C:\Users\BKSH\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


252/252 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.3865 - loss: 1.7210 - val_accuracy: 0.6066 - val_loss: 1.1383
Epoch 2/10
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6092 - loss: 1.1668 - val_accuracy: 0.6488 - val_loss: 1.0364
Epoch 3/10
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6509 - loss: 1.0398 - val_accuracy: 0.6622 - val_loss: 0.9903
Epoch 4/10
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6623 - loss: 0.9955 - val_accuracy: 0.6602 - val_loss: 0.9858
Epoch 5/10
252/252 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6873 - loss: 0.9243 - val_accuracy: 0.6721 - val_loss: 0.9572
Epoch 6/10
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6956 - loss: 0.8927 - val_accuracy: 0.6662 - val_loss: 0.9598
Epoch 7/10
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7018 - loss: 0.8708 - val_accuracy: 0.6796 - val_loss: 0.9497
Epoch 8/10
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7206 - loss: 0.8308 - val_accuracy: 0.6662 - val_

# ـــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــ

# • Evaluate the Models:

In [60]:
# • Evaluating on Test Data:

#TF-IDF
tfidf_loss, tfidf_accuracy = tfidf_model.evaluate(X_test_tfidf, y_test_onehot)
print(f"TF-IDF Test Accuracy: {tfidf_accuracy * 100:.2f}%")

#Word2Vec
w2v_loss, w2v_accuracy = w2v_model.evaluate(X_test_w2v, y_test_onehot)
print(f"Word2Vec Test Accuracy: {w2v_accuracy * 100:.2f}%")


#Predicting the probabilities for the TF-IDF model
y_pred_tfidf = tfidf_model.predict(X_test_tfidf)

#Converting the probabilities to class labels
y_pred_tfidf_labels = y_pred_tfidf.argmax(axis=1)

#Converting one-hot test labels to class labels
y_test_labels = y_test_onehot.argmax(axis=1)

#Predicting the probabilities for the Word2Vec model
y_pred_w2v = w2v_model.predict(X_test_w2v)

#Converting the probabilities to class labels
y_pred_w2v_labels = y_pred_w2v.argmax(axis=1)

print("FNN (TF-IDF) Classification Report")
print(classification_report(y_test_labels, y_pred_tfidf_labels))

print("FNN (Word2Vec) Classification Report")
print(classification_report(y_test_labels, y_pred_w2v_labels))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5979 - loss: 2.0533 
TF-IDF Test Accuracy: 59.86%
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6836 - loss: 0.9471 
Word2Vec Test Accuracy: 67.41%
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
FNN (TF-IDF) Classification Report
              precision    recall  f1-score   support

           0       0.56      0.60      0.58       276
           1       0.93      0.86      0.89       259
           2       0.43      0.50      0.46       268
           3       0.63      0.59      0.61       250
           4       0.58      0.66      0.62       307
           5       0.39      0.36      0.37       258
           6       0.51      0.38      0.43       201
           7       0.83      0.85      0.84       194

    accuracy                           0.60      2013
   macro avg       0.61      0.60      0.60      2013
weighted avg       0.60      0.60      0.60      2013

FNN (Word2Vec) Classificat

# ـــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــ

# • Tokenize the Text:

In [65]:
#Defining the tokenizer
tokenizer = Tokenizer(oov_token="<OOV>")  
tokenizer.fit_on_texts(data['normalized_TWEET'])

#Converting the text to sequences
sequences = tokenizer.texts_to_sequences(data['normalized_TWEET'])

#Getting the vocabulary size
vocab_size = len(tokenizer.word_index) + 1 
print(f"Vocabulary size: {vocab_size}")

Vocabulary size: 33671


# ـــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــ

# • Pad the Sequences:

In [69]:
#Defining the maximum sequence length
max_length = 100  

#Padding the sequences
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')

# ـــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــ

# • Encode the Labels:

In [72]:
#Encoding the labels
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(data['LABEL'])

#Converting to one-hot encoding
y_onehot = to_categorical(y_encoded)

# ـــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــ

# • Train-Test Split:

In [76]:
X_train, X_test, y_train, y_test = train_test_split(
    padded_sequences, y_onehot, test_size=0.2, random_state=42
)

# ـــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــ

# • Load Pre-trained Word2Vec Embeddings:

In [79]:
# • Creating an Embedding Matrix:

#Initializing the embedding matrix
embedding_dim = word2vec_model.vector_size  # Dimension of Word2Vec embeddings
embedding_matrix = np.zeros((vocab_size, embedding_dim))

#Populating the embedding matrix
for word, i in tokenizer.word_index.items():
    if word in word2vec_model.wv:
        embedding_matrix[i] = word2vec_model.wv[word]

#Defining an Embedding Layer:
embedding_layer = Embedding(
    input_dim=vocab_size,
    output_dim=embedding_dim,
    weights=[embedding_matrix],
    input_length=max_length,
    trainable=False  
)

C:\Users\BKSH\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


# ـــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــ

# • Define the LSTM Model:

In [83]:
#Defining the LSTM model
def create_lstm_model():
    model = Sequential([
        embedding_layer,  # Pre-trained Word2Vec embeddings
        Bidirectional(LSTM(128, return_sequences=False)),  # BiLSTM layer
        Dropout(0.3),  # Dropout for regularization
        Dense(64, activation='relu'),  # Fully connected layer
        Dropout(0.3),
        Dense(y_train.shape[1], activation='softmax')  # Output layer
    ])
    
    #Compiling the model
    model.compile(optimizer='adam', 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
    
    return model

# ـــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــ

# • Train the Model:

In [86]:
#Creating the LSTM model
lstm_model = create_lstm_model()

#Training the model
history_lstm = lstm_model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=10,
    batch_size=32,
    verbose=1
)

Epoch 1/10
252/252 ━━━━━━━━━━━━━━━━━━━━ 22s 71ms/step - accuracy: 0.4358 - loss: 1.5953 - val_accuracy: 0.6503 - val_loss: 1.0060
Epoch 2/10
252/252 ━━━━━━━━━━━━━━━━━━━━ 18s 70ms/step - accuracy: 0.6656 - loss: 1.0091 - val_accuracy: 0.6647 - val_loss: 0.9510
Epoch 3/10
252/252 ━━━━━━━━━━━━━━━━━━━━ 16s 64ms/step - accuracy: 0.6949 - loss: 0.9021 - val_accuracy: 0.6841 - val_loss: 0.9028
Epoch 4/10
252/252 ━━━━━━━━━━━━━━━━━━━━ 19s 75ms/step - accuracy: 0.7360 - loss: 0.7813 - val_accuracy: 0.6791 - val_loss: 0.9012
Epoch 5/10
252/252 ━━━━━━━━━━━━━━━━━━━━ 19s 74ms/step - accuracy: 0.7387 - loss: 0.7560 - val_accuracy: 0.6965 - val_loss: 0.8956
Epoch 6/10
252/252 ━━━━━━━━━━━━━━━━━━━━ 18s 70ms/step - accuracy: 0.7762 - loss: 0.6578 - val_accuracy: 0.6890 - val_loss: 0.9191
Epoch 7/10
252/252 ━━━━━━━━━━━━━━━━━━━━ 19s 77ms/step - accuracy: 0.7931 - loss: 0.6067 - val_accuracy: 0.6850 - val_loss: 0.9062
Epoch 8/10
252/252 ━━━━━━━━━━━━━━━━━━━━ 17s 66ms/step - accuracy: 0.7989 - loss: 0.5710 - 

# ـــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــ

# • Evaluate the Model:

In [89]:
# • Evaluating on Test Data:
loss, accuracy = lstm_model.evaluate(X_test, y_test)
print(f"LSTM Test Accuracy: {accuracy * 100:.2f}%")

#LSTM Model Predictions
y_pred_lstm = lstm_model.predict(X_test)
y_pred_lstm_labels = y_pred_lstm.argmax(axis=1)
y_test_lstm_labels = y_test.argmax(axis=1)

print("LSTM Classification Report")
print(classification_report(y_test_lstm_labels, y_pred_lstm_labels))

63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.6854 - loss: 1.0653
LSTM Test Accuracy: 67.56%
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step  
LSTM Classification Report
              precision    recall  f1-score   support

           0       0.76      0.66      0.71       276
           1       0.94      0.93      0.93       259
           2       0.57      0.55      0.56       268
           3       0.79      0.57      0.66       250
           4       0.64      0.84      0.73       307
           5       0.53      0.45      0.49       258
           6       0.41      0.59      0.49       201
           7       0.92      0.80      0.85       194

    accuracy                           0.68      2013
   macro avg       0.70      0.67      0.68      2013
weighted avg       0.69      0.68      0.68      2013



# ـــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــ

# <center> **The End** </center> 

# ـــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــ